## Creating Data: Compute Measures for Enrollment Panel Data

from White Flight in Higher Education? The Case of Hispanic-Serving Institutions

by Laura Hamilton, Charlie Eaton, and Simon Cheng

In [1]:
qui {
    use hdef19802019, clear

**Create separate unitid for Benjamin Franklin Institute for when it goes from Public to Private
    **Correct IPEDS reporting error to note it is private in 2016 & 2017
replace unitid=unitid+1000000 if unitid==165884 & year>2007
    replace control=2 if unitid==1165884
    
*replace Boricuo college Latinx enrollment as missing for years 1993 & 1994
    * when reporting error reduced % Latinx from 95% to 0%
replace tefhispt=. if unitid==189413 & year<1995 & year>1992
    
*keep if teftotlt>100 & ftfeftotlt<. & control<3
    
foreach var in efbkaat efhispt efwhitt efaiant efasiat {
quietly    gen pctt`var'= t`var' / teftotlt * 100
quietly gen lnftf`var'=log(ftf`var')
    quietly gen lnt`var'=log(t`var')
}
foreach var in cpopam cpopas cpopbl cpopwh cpophi populationam populationas populationbl populationhi populationwh {
    replace `var'=0 if `var'==.
        gen ln`var'=log(`var'+1)
    }

quietly gen lnftfeftotlt=log(ftfeftotlt)
replace pcttefhispt= Latinx_enr / tot_enroll * 25 if pcttefhispt==.

   
quietly gen hispserve=0 if pcttefhispt!=.
quietly replace hispserve=1 if pcttefhispt>=25 & pcttefhispt<.

xtset unitid year

**create var for year in which event occurs observation year of event for subject**
by unitid: gen transition=year if hispserv==1 & l.hispserv==0

**create var for year in which event occurs in all observations for all years for subject**
by unitid: egen hispserveyear = min(transition)

**generate a var for the number of years relative to the year of the event (e.g. + or - 2 years)

gen hispserveyeardiff= year-hispserveyear
}
bysort unitid: egen mcontrol=mode(control), minmode

drop if year<1990
keep if mcontrol==1 & iclevel==1
save d_hsi_white_enrollment_final3, replace

         Generating missing values for the modes in these groups. Use option
         missing to treat missing values as a category.
(52 missing values generated)
(27,941 observations deleted)
(226,630 observations deleted)
file d_hsi_white_enrollment_final3.dta saved
